In [1]:
import mbuild as mb
from mbuild.lib.recipes import TiledCompound
import numpy as np
import warnings
from random import sample
from copy import deepcopy
import openbabel as ob
from openbabel import pybel
from copy import deepcopy
# from slab_builder import *
warnings.simplefilter("ignore")

/home/robert/mambaforge/envs/simulations/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/robert/mambaforge/envs/simulations/lib/python3.11/site-packages/mdtraj/geometry/order.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/home/robert/mambaforge/envs/simulations/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/robert/mambaforge/envs/simulations/lib/python3.11/site-packages/pkg_resources/__init__.p

In [2]:
slab = mb.load('../slab-assembly/minimized-ase-slab.mol2')
slab.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
ligands = mb.load("lammps-testing/bonded_structures/structures/additive-bonded-at-ends.mol2")
ligands.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
ligand2 = mb.load('lammps-testing/bonded_structures/structures/additive-bonded-at-base.mol2')
ligand2.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
parmed_structure = ligands.to_parmed()

def find_coo_groups(parmed_structure):
    coo_indices = []
    
    for atom in parmed_structure.atoms:
        if atom.name == 'C':
            oxygen_neighbors = [a for a in atom.bond_partners if a.name == 'O']
            
            if len(oxygen_neighbors) == 2:
                coo_indices.append(atom.idx)
                
    return coo_indices

coo_indices = find_coo_groups(parmed_structure)


In [6]:
coo1_index = coo_indices[0]
coo2_index = coo_indices[1]
ligand_system = mb.load("lammps-testing/bonded_structures/structures/additive-bonded-at-ends.mol2")

coo1_coord = ligand_system[coo1_index].pos
coo2_coord = ligand_system[coo2_index].pos

slab1 = mb.load('../slab-assembly/minimized-ase-slab.mol2')
slab2 = mb.load('../slab-assembly/minimized-ase-slab.mol2')

slab1.translate([0, 0, -2.3])  # One slab at z = -1.1 nm
slab2.translate([0, 0, 1.5])   # Another slab at z = 1.1 nm

# Translate the molecule to center it between slabs
ligand_system.translate([0, 0, 0])

# Further translation and rotation here to align the end groups with the slabs
# Assuming slab1 is at z = -1.1 nm and slab2 is at z = 1.1 nm
midpoint = (slab1.center[2] + slab2.center[2]) / 2
ligand_center = ligand_system.center[2]

# Translate ligand system to the midpoint
z_translation = midpoint - ligand_center
ligand_system.translate([0, 0, z_translation])

# Compute the vector along which to align the ligand
alignment_vector = np.array([0, 0, 1])  # z-axis in this case

# Compute the current vector connecting the COO groups
current_vector = coo2_coord - coo1_coord
current_vector = current_vector / np.linalg.norm(current_vector)  # Normalize the vector

# Compute the axis and angle for rotation
rotation_axis = np.cross(current_vector, alignment_vector)
rotation_angle = np.arccos(np.dot(current_vector, alignment_vector))

# Perform the rotation
ligand_system.rotate(rotation_angle, around=rotation_axis)

ligand_system.translate([2,0.9,0])

In [7]:
system = mb.Compound(subcompounds=[slab1, slab2, ligand_system])
system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
%pwd

'/home/robert/qd-crosslink/swelling'

In [9]:
ligand_system.save("additive-test.mol2")

In [11]:
slab1.save("slab1-test.mol2")
slab2.save("slab2-test.mol2")

In [9]:
for Zn in slab:
    if Zn.element.symbol == 'Zn' and Zn.pos[2] > 1.5:
        # port = mb.Port(anchor=Zn, orientation=[0,0,1], separation=0.05)
        # slab.add(port)
            if Zn.pos[1] 
        print(Zn.pos)

[ 0.42326    -0.06774     1.50055003]
[0.42326999 0.47126001 1.50055003]
[0.42326999 1.01025999 1.50055003]
[0.42326999 1.54926002 1.50055003]
[0.42326    2.08825994 1.50055003]
[0.42326    2.62725997 1.50055003]
[ 1.18561995 -0.06778     1.50058997]
[1.18561995 0.47121999 1.50058997]
[1.18561995 1.01022005 1.50058997]
[1.18561995 1.54921997 1.50059998]
[1.18561995 2.08821988 1.50059998]
[1.18561995 2.62721992 1.50058997]
[ 1.94795001 -0.06777     1.50059998]
[1.94795001 0.47123    1.50059998]
[1.94795001 1.01022995 1.50059998]
[1.94795001 1.54922998 1.50059998]
[1.94795001 2.08822989 1.50059998]
[1.94795001 2.62722993 1.50059998]
[ 2.71021008 -0.06776     1.50062001]
[2.71021008 0.47123    1.50062001]
[2.71021008 1.01022995 1.50062001]
[2.71021008 1.54922998 1.50062001]
[2.71021008 2.08822989 1.50062001]
[2.71021008 2.62722993 1.50062001]
[ 3.47238994 -0.06775     1.50060999]
[3.47238994 0.47125    1.50060999]
[3.47238994 1.01024997 1.50060999]
[3.47238994 1.54925001 1.50060999]
[3.47

In [ ]:
zinc_positions = {'x':}
for Zn in slab:
    if Zn.element.symbol == 'Zn':
        zinc_positions['x']